У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [404]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from src.utils import predict_and_plot

In [405]:
raw_df = pd.read_csv("customer_segmentation_train.csv")
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [406]:
X_cols = raw_df.columns[1:-1].tolist()
y_cols = raw_df.columns[-1]

In [407]:
imput_unknown_cols = ['Ever_Married', 'Graduated', 'Profession', 'Var_1']
imput_median_cols = ['Work_Experience', 'Family_Size']

numeric_cols = raw_df[X_cols].select_dtypes('number').columns.tolist()
categorical_cols = raw_df[X_cols].select_dtypes('object').columns.tolist()

In [408]:
X_train, X_test, y_train, y_test = train_test_split(raw_df[X_cols], raw_df[y_cols], test_size=0.2, random_state=42, stratify=raw_df[y_cols])

In [409]:
# perform imputation outside of pipeline to be able to use data in resampling processes
numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

X_train[numeric_cols] = numeric_imputer.fit_transform(X_train[numeric_cols])
X_train[categorical_cols] = categorical_imputer.fit_transform(X_train[categorical_cols])

In [410]:
numeric_transformer = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, numeric_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ]
)
numeric_preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, numeric_cols),
    ]
)

# the model with pre-encoded categorical data for SMOTETomek resampling
ovr_model_pipeline = Pipeline([
    ('prepocessor', preprocessor),
    ('model', OneVsRestClassifier(LogisticRegression(solver='liblinear', penalty='l1', C=2.0)))
])
numeric_ovr_model_pipeline = Pipeline([
    ('prepocessor', numeric_preprocessor),
    ('model', OneVsRestClassifier(LogisticRegression(solver='liblinear', penalty='l1', C=2.0)))
])

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
smotenc = SMOTENC(random_state=0, categorical_features=[0, 1, 3, 4, 6, 8])
smotomek = SMOTETomek(random_state=0)

# manually create dataframe for SMOTETomek with encoded categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(X_train[categorical_cols])
categorical_enc_cols = encoder.get_feature_names_out(categorical_cols)
X_train_encoded = X_train.drop(columns=categorical_cols)
X_train_encoded[categorical_enc_cols] = categorical_encoded

X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)
X_train_smotomek, y_train_smotomek = smotomek.fit_resample(X_train_encoded, y_train)

Segmentation
D    1814
A    1578
C    1576
B    1486
Name: count, dtype: int64

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [412]:
ovr_model_pipeline.fit(X_train, y_train)
ovr_preds_origin = ovr_model_pipeline.predict(X_train)
print("Model metrics on imbalanced data:")
print(classification_report(y_train, ovr_preds_origin))

Model metrics on imbalanced data:
              precision    recall  f1-score   support

           A       0.45      0.48      0.47      1578
           B       0.41      0.19      0.26      1486
           C       0.50      0.64      0.57      1576
           D       0.64      0.73      0.68      1814

    accuracy                           0.52      6454
   macro avg       0.50      0.51      0.49      6454
weighted avg       0.51      0.52      0.50      6454



In [413]:
print("Model metrics on data balanced by SMOTENC:")
ovr_model_pipeline.fit(X_train_smotenc, y_train_smotenc)
ovr_preds_smotenc = ovr_model_pipeline.predict(X_train_smotenc)
print(classification_report(y_train_smotenc, ovr_preds_smotenc))

Model metrics on data balanced by SMOTENC:
              precision    recall  f1-score   support

           A       0.45      0.50      0.48      1814
           B       0.43      0.26      0.32      1814
           C       0.51      0.61      0.56      1814
           D       0.62      0.68      0.64      1814

    accuracy                           0.51      7256
   macro avg       0.50      0.51      0.50      7256
weighted avg       0.50      0.51      0.50      7256



In [414]:
print("Model metrics on data balanced by SMOTETomek:")
numeric_ovr_model_pipeline.fit(X_train_smotomek, y_train_smotomek)
ovr_preds_smotomek = numeric_ovr_model_pipeline.predict(X_train_smotomek)
print(classification_report(y_train_smotomek, ovr_preds_smotomek))

Model metrics on data balanced by SMOTETomek:
              precision    recall  f1-score   support

           A       0.37      0.31      0.34      1564
           B       0.30      0.14      0.19      1623
           C       0.38      0.56      0.45      1646
           D       0.55      0.69      0.61      1603

    accuracy                           0.42      6436
   macro avg       0.40      0.42      0.40      6436
weighted avg       0.40      0.42      0.40      6436



**Висновок**

- Обрав метрику f1 як основну.
- Найкраща модель з SMOTENC балансуванням. Так як дещо покращилась класифікація класів A, B.
- Суттєвої різниці немає. Дисбаланс даних не такий вже і великий, тому балансування не дає значних покращень. Або треба більше даних, більше колонок для побудови точнішої моделі.